In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split

### Map words to glove embeddings 

In [2]:
embeddings_dims = 50

with open('../models/embeddings/glove.6B.'+str(embeddings_dims)+'d.txt','rb') as f:
    word_entries = f.readlines()
    
glove_weights = np.zeros((len(word_entries), embeddings_dims))
words = []
for i, entry in enumerate(word_entries):
    word_weights = entry.split()
    word = word_weights[0]
    words.append(word)
    weights = word_weights[1:]
    glove_weights[i] = np.array([float(w) for w in weights])
word_vocab = [w.decode("utf-8") for w in words]
word2vec = dict(zip(word_vocab, glove_weights))

### Import data

In [64]:
df = pd.read_csv("../nlp_dataset/raw/fake_or_real_news.csv")
df = df.set_index("Unnamed: 0") 
df.head()

,title,text,label
Unnamed: 0,,,
8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


#### Text preprocessing

In [65]:
df.title = df.title.str.lower()
df.text = df.text.str.lower()
df.title = df.title.str.replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
df.text = df.text.str.replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
df.title = df.title.str.replace(r'\.\.+','.') #replace multple periods with a single one
df.text = df.text.str.replace(r'\.\.+','.') #replace multple periods with a single one
df.title = df.title.str.replace(r'\.',' . ') #replace periods with a single one
df.text = df.text.str.replace(r'\.',' . ') #replace multple periods with a single one
df.title = df.title.str.replace(r'\s\s+',' ') #replace multple white space with a single one
df.text = df.text.str.replace(r'\s\s+',' ') #replace multple white space with a single one
df.title = df.title.str.strip() 
df.text = df.text.str.strip() 

def tokenize(text):
    text = text.split()
    return text
    
def remove_stopwords(tokens):
    new_tokens =  [token for token in tokens if token not in stopwords.words('english')]
    return new_tokens

def transform(tokens_list, word2vec):
    embeddings_average = []
    for text in tokens_list:
        emb_mean = np.mean([word2vec[word] for word in text if word in word2vec] or [np.zeros(embeddings_dims)], axis=0)
        embeddings_average.append(emb_mean)
    return np.array(embeddings_average)

### Build features

In [66]:
df['text_tokens'] = df['text'].apply(tokenize)
df['title_tokens'] = df['title'].apply(tokenize).apply(remove_stopwords)
X = transform(df['text_tokens'], word2vec)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.2, random_state=53)

### Fit and evaluate models

In [62]:
names = ["Nearest Neighbors","Linear SVM", "RBF SVM",
         "Decision Tree", "Random Forest", "Neural Net", 'ExtraTreesClassifier']

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    ExtraTreesClassifier(n_estimators=200)]

In [63]:
for name, clf in zip(names, classifiers):
    pred = clf.fit(X_train, y_train).predict(X_test)
    score = metrics.accuracy_score(y_test, pred)
    print("Classifier {0}, Accuracy {1}".format(name, score))

Classifier Nearest Neighbors, Accuracy 0.8326756116811366
Classifier Linear SVM, Accuracy 0.7758484609313339
Classifier RBF SVM, Accuracy 0.8800315706393055
Classifier Decision Tree, Accuracy 0.7561168113654302
Classifier Random Forest, Accuracy 0.760852407261247
Classifier Neural Net, Accuracy 0.7876874506708761
Classifier ExtraTreesClassifier, Accuracy 0.8531965272296764
